# Imports

In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder

import gc
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

# Loading Data

In [3]:
train = pd.read_csv('train.csv')

In [44]:
building = pd.read_csv('building_metadata.csv')

In [5]:
weather_train = pd.read_csv('weather_train.csv')

# Reduce Memory Usage

In [7]:
## Function to reduce the DF size
def reduce_mem_usage(df, use_float16=False):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        if is_datetime(df[col]) or is_categorical_dtype(df[col]):
            # skip datetime type or categorical type
            continue
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if use_float16 and c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

# 1. Weather_train: Imputations for Weather Info

Set timestamp as index.\
Add more features for timeseries. Would benifit for later missing values imputation.\
Assign data types to new features.

In [8]:
def add_times_series_feature(df):
    df= df.set_index('timestamp')
    df.index = pd.to_datetime(df.index)
    df['hour']=df.index.hour
    df['day'] = df.index.day
    df['weekday']=df.index.weekday
    df['month']=df.index.month
    df=df.reset_index()
    return df

In [9]:
weather_train_processed=add_times_series_feature(weather_train)

In [10]:
weather_train_processed.head()

,timestamp,site_id,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,hour,day,weekday,month
0,2016-01-01 00:00:00,0,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,0,1,4,1
1,2016-01-01 01:00:00,0,24.4,NaN,21.1,-1.0,1020.2,70.0,1.5,1,1,4,1
2,2016-01-01 02:00:00,0,22.8,2.0,21.1,0.0,1020.2,0.0,0.0,2,1,4,1
3,2016-01-01 03:00:00,0,21.1,2.0,20.6,0.0,1020.1,0.0,0.0,3,1,4,1
4,2016-01-01 04:00:00,0,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6,4,1,4,1


In [11]:
weather_train_processed.dtypes

timestamp             datetime64[ns]
site_id                        int64
air_temperature              float64
cloud_coverage               float64
dew_temperature              float64
precip_depth_1_hr            float64
sea_level_pressure           float64
wind_direction               float64
wind_speed                   float64
hour                           int64
day                            int64
weekday                        int64
month                          int64
dtype: object

For weather information, \
cloud_coverage	   43.655% missing, {0,1,2,...,9} \
precip_depth_1_hr	18.545% missing \
sea_level_pressure : site 5 has no record\
dew_temperature: site 10 has pretty low dew temperature

## 1.1 cloud_coverage, precip_depth_1_hr, sea_leverl_pressure, wind_direction, wind_speed

Those variables are missing on consecutive days. Fill missing values by the day of the month mean.\
Notice that cloud_coverage are actually categorical values from 0 ,1,2,...,9. Use np.rint to round up to an interger

## 1.2 air_temperature, dew_temperature

Theoretically, they are highly correlated to each other. \
<span style='color:red'>air_temperature must be higher than dew_temperature.</span>

In [12]:
weather_train_processed.head()

,timestamp,site_id,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,hour,day,weekday,month
0,2016-01-01 00:00:00,0,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,0,1,4,1
1,2016-01-01 01:00:00,0,24.4,NaN,21.1,-1.0,1020.2,70.0,1.5,1,1,4,1
2,2016-01-01 02:00:00,0,22.8,2.0,21.1,0.0,1020.2,0.0,0.0,2,1,4,1
3,2016-01-01 03:00:00,0,21.1,2.0,20.6,0.0,1020.1,0.0,0.0,3,1,4,1
4,2016-01-01 04:00:00,0,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6,4,1,4,1


In [13]:
weather_train_processed.dtypes

timestamp             datetime64[ns]
site_id                        int64
air_temperature              float64
cloud_coverage               float64
dew_temperature              float64
precip_depth_1_hr            float64
sea_level_pressure           float64
wind_direction               float64
wind_speed                   float64
hour                           int64
day                            int64
weekday                        int64
month                          int64
dtype: object

In [14]:
def weather_info_imputation(weather_df):
    """
    Impute missing values from weather dataset by the mean value of the day of the month (forward fill)
    """

    weather_df = weather_df.set_index(['site_id','month','day'])
    cloud_coverage_filler = np.rint(weather_df.groupby(['site_id','month','day'])['cloud_coverage'].mean())
    cloud_coverage_filler = pd.DataFrame(cloud_coverage_filler.fillna(method='ffill'),columns=['cloud_coverage'])
    weather_df.update(cloud_coverage_filler,overwrite=False)

    precip_depth_filler = weather_df.groupby(['site_id','month','day'])['precip_depth_1_hr'].mean()
    precip_depth_filler = pd.DataFrame(precip_depth_filler.fillna(method='ffill'),columns=['precip_depth_1_hr'])
    weather_df.update(precip_depth_filler,overwrite=False)

    sea_filler = weather_df.groupby(['site_id','month','day'])['sea_level_pressure'].mean()
    sea_filler = pd.DataFrame(sea_filler.fillna(method='ffill'),columns=['sea_level_pressure'])
    weather_df.update(sea_filler,overwrite=False)
    
    wind_direction_filler =  pd.DataFrame(weather_df.groupby(['site_id','month','day'])['wind_direction'].mean(),columns=['wind_direction'])
    weather_df.update(wind_direction_filler,overwrite=False)

    wind_speed_filler =  pd.DataFrame(weather_df.groupby(['site_id','month','day'])['wind_speed'].mean(),columns=['wind_speed'])
    weather_df.update(wind_speed_filler,overwrite=False)
    
    
   
    air_temperature_filler = weather_df.groupby(['site_id','month','day'])['air_temperature'].mean()
    air_temperature_filler = pd.DataFrame(air_temperature_filler.fillna(method='ffill'),columns=['air_temperature'])
    weather_df.update( air_temperature_filler,overwrite=False)
                                                                           
    dew_temperature_filler = weather_df.groupby(['site_id','month','day'])[ 'dew_temperature'].mean()
    dew_temperature_filler = pd.DataFrame(dew_temperature_filler.fillna(method='ffill'),columns=['dew_temperature'])                                                                      
    weather_df.update( dew_temperature_filler,overwrite=False)
    
    wrong_temp=weather_df[weather_df['air_temperature'] < weather_df['dew_temperature']][['air_temperature','dew_temperature']]
    for i in range(len(wrong_temp)):
        wrong_temp['air_temperature'].iloc[i]=wrong_temp['dew_temperature'].iloc[i]
       
    wrong_temp=wrong_temp.drop('dew_temperature',axis=1)
    weather_df.update(wrong_temp.values,overwrite=False)
    
    weather_df=weather_df.reset_index()
    weather_df=weather_df.drop(['hour','day','weekday','month'],axis=1)
    
    
    return weather_df


In [15]:
weather_train_processed=weather_info_imputation(weather_train_processed)

In [16]:
weather_train_processed.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2016-01-01 00:00:00,25.0,6.0,20.0,-0.173913,1019.7,0.0,0.0
1,0,2016-01-01 01:00:00,24.4,4.0,21.1,-1.000000,1020.2,70.0,1.5
2,0,2016-01-01 02:00:00,22.8,2.0,21.1,0.000000,1020.2,0.0,0.0
3,0,2016-01-01 03:00:00,21.1,2.0,20.6,0.000000,1020.1,0.0,0.0
4,0,2016-01-01 04:00:00,20.0,2.0,20.0,-1.000000,1020.0,250.0,2.6


In [17]:
weather_train_processed.columns

Index(['site_id', 'timestamp', 'air_temperature', 'cloud_coverage',
       'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure',
       'wind_direction', 'wind_speed'],
      dtype='object')

In [18]:
weather_train_na_vars = [var for var in weather_train_processed.columns if weather_train_processed[var].isnull().mean()>0]
weather_train_na_vars 

[]

All the weather variables for training set have been imputated.Reset index for later merging.

In [19]:
gc.collect()

325

# 2. building: Imputations for floor_count,year_built

floor_count: missing percentage	82.7% \
year_built: missing precentage 60%\
corr(square_feet,floor_count)=0.58 \
Here, only computate floor_count by building_id. Drop floor_count and keep sauqre_feet.

In [20]:
building.head()

,site_id,building_id,primary_use,square_feet,year_built,floor_count
0,0,0,Education,7432,2008.0,NaN
1,0,1,Education,2720,2004.0,NaN
2,0,2,Education,5376,1991.0,NaN
3,0,3,Education,23685,2002.0,NaN
4,0,4,Education,116607,1975.0,NaN


In [819]:
building_no_floor['year_built'].isnull().sum() #774 missing year_built for total 1449 rows

774

There is only one unique year_built value for the same building_id.

In [21]:
building.groupby(['building_id'])['year_built'].value_counts(dropna=False)

building_id  year_built
0            2008.0        1
1            2004.0        1
2            1991.0        1
3            2002.0        1
4            1975.0        1
5            2000.0        1
6            1981.0        1
7            1989.0        1
8            2003.0        1
9            2010.0        1
10           1991.0        1
11           1968.0        1
12           1999.0        1
13           2000.0        1
14           2013.0        1
15           1974.0        1
16           1996.0        1
17           1980.0        1
18           1996.0        1
19           2004.0        1
20           1977.0        1
21           1990.0        1
22           1996.0        1
23           1985.0        1
24           2001.0        1
25           1969.0        1
26           2005.0        1
27           1999.0        1
28           2016.0        1
29           2006.0        1
                          ..
1419         1975.0        1
1420         1975.0        1
1421         NaN   

In [22]:
def building_feature_engineering(df):
    """
    Drop floor_count;Fill the missing year_built values by the year_built for the same building_id.
    Encode primary_use
    """
    # 1-Drop floor_count due to highly correlated to square_feet
    df=df.drop('floor_count',axis=1)
    
    #2-Imputate year_built
    df=df.set_index('building_id')
    df_filled=pd.DataFrame(df.groupby('building_id')['year_built'].agg(np.mean).transform(lambda x:x.fillna(method='ffill')),columns=['year_built'])
    df.update(df_filled,overwrite=True)
    df=df.reset_index()
    
    #3 Encode categorical variable--primary_use
    le = LabelEncoder()
    df['primary_use'] = le.fit_transform(df['primary_use'])
    
    return df

In [23]:
building_processed = building_feature_engineering(building)

In [24]:
building_processed

,building_id,site_id,primary_use,square_feet,year_built
0,0,0,0,7432,2008.0
1,1,0,0,2720,2004.0
2,2,0,0,5376,1991.0
3,3,0,0,23685,2002.0
4,4,0,0,116607,1975.0
5,5,0,0,8000,2000.0
6,6,0,4,27926,1981.0
7,7,0,0,121074,1989.0
8,8,0,0,60809,2003.0
9,9,0,6,27000,2010.0


In [25]:
building_processed['year_built'].isna().any()

False

## 3. Merge Datasets by Foreign Keys

In [26]:
# Merge train, weather_train via foreign key from building 
# train---building_id----building
# train---site_id,timestamp---weather_train
train_processed = train.merge(building_processed,left_on = ['building_id'], right_on=['building_id'],how='left')

In [27]:
train_processed.dtypes

building_id        int64
meter              int64
timestamp         object
meter_reading    float64
site_id            int64
primary_use        int32
square_feet        int64
year_built       float64
dtype: object

<span style='color:red'>Need to change the timestamp type from datetime to str. Later should be processed again.</span>

In [28]:
train_processed['timestamp']=train_processed['timestamp'].astype(str)
weather_train_processed['timestamp']=weather_train_processed['timestamp'].astype(str)

In [29]:
train_processed = train_processed.merge(weather_train_processed, left_on=['site_id','timestamp'],right_on=['site_id','timestamp'],how='left')

In [30]:
train_processed.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,2016-01-01 00:00:00,0.0,0,0,7432,2008.0,25.0,6.0,20.0,-0.173913,1019.7,0.0,0.0
1,1,0,2016-01-01 00:00:00,0.0,0,0,2720,2004.0,25.0,6.0,20.0,-0.173913,1019.7,0.0,0.0
2,2,0,2016-01-01 00:00:00,0.0,0,0,5376,1991.0,25.0,6.0,20.0,-0.173913,1019.7,0.0,0.0
3,3,0,2016-01-01 00:00:00,0.0,0,0,23685,2002.0,25.0,6.0,20.0,-0.173913,1019.7,0.0,0.0
4,4,0,2016-01-01 00:00:00,0.0,0,0,116607,1975.0,25.0,6.0,20.0,-0.173913,1019.7,0.0,0.0


# 4. Train: Variables in Train preprocessing

Training Set Features:

Training Set Features:\
Add hour,weekday features.\
Remove outlier building_id=1099----Based on EDA, building 1099 would draw a big influnce on the abnormality on meter_reading timeseries curve.\
square_feet---log1p

Training Set Target:\
meter_reading---log1p

In [31]:
train_processed.dtypes

building_id             int64
meter                   int64
timestamp              object
meter_reading         float64
site_id                 int64
primary_use             int32
square_feet             int64
year_built            float64
air_temperature       float64
cloud_coverage        float64
dew_temperature       float64
precip_depth_1_hr     float64
sea_level_pressure    float64
wind_direction        float64
wind_speed            float64
dtype: object

In [32]:
train_processed.columns

Index(['building_id', 'meter', 'timestamp', 'meter_reading', 'site_id',
       'primary_use', 'square_feet', 'year_built', 'air_temperature',
       'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr',
       'sea_level_pressure', 'wind_direction', 'wind_speed'],
      dtype='object')

In [33]:
def train_engineering(df):
    # Remove Outlier building_id=1099
    building_1099=df[df['building_id']==1099]
    df=df[df['building_id']!=1099]
    
    # Change timestamp to datetime type and add time features
    df= df.set_index('timestamp')
    df.index = pd.to_datetime(df.index)
    df['hour'] = df.index.day
    df['day'] = df.index.day
    df['weekday']=df.index.weekday

    # For more accurate numerical calculations
    df['year_built'] = df['year_built']-1900
    df['square_feet'] = df['square_feet']

    # Target

    df=df.reset_index()
    df['meter_reading'] = np.log1p(df['meter_reading'])
    
    df=df.reset_index()
    
    return df

In [34]:
train_processed = train_engineering(train_processed)


In [35]:
train_processed.head()

,index,timestamp,building_id,meter,meter_reading,site_id,primary_use,square_feet,year_built,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,hour,day,weekday
0,0,2016-01-01,0,0,0.0,0,0,7432,108.0,25.0,6.0,20.0,-0.173913,1019.7,0.0,0.0,1,1,4
1,1,2016-01-01,1,0,0.0,0,0,2720,104.0,25.0,6.0,20.0,-0.173913,1019.7,0.0,0.0,1,1,4
2,2,2016-01-01,2,0,0.0,0,0,5376,91.0,25.0,6.0,20.0,-0.173913,1019.7,0.0,0.0,1,1,4
3,3,2016-01-01,3,0,0.0,0,0,23685,102.0,25.0,6.0,20.0,-0.173913,1019.7,0.0,0.0,1,1,4
4,4,2016-01-01,4,0,0.0,0,0,116607,75.0,25.0,6.0,20.0,-0.173913,1019.7,0.0,0.0,1,1,4


In [36]:
train_processed=train_processed.drop('index',axis=1)

In [37]:
train_processed.columns

Index(['timestamp', 'building_id', 'meter', 'meter_reading', 'site_id',
       'primary_use', 'square_feet', 'year_built', 'air_temperature',
       'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr',
       'sea_level_pressure', 'wind_direction', 'wind_speed', 'hour', 'day',
       'weekday'],
      dtype='object')

In [38]:
category = ['hour','meter','weekday','primary_use','site_id','building_id']
for col in category :
    train_processed[col] = train_processed[col].astype('category')

In [39]:
target = train_processed['meter_reading']
features = train_processed.drop(['meter_reading','timestamp'], axis = 1)

In [42]:
features.to_csv(r'C:\Users\syxsw\MLProjects\BuildingEnergy\ASHRAE---Great-Energy-Predictor-III-master\train_features.csv')

In [43]:
target.to_csv(r'C:\Users\syxsw\MLProjects\BuildingEnergy\ASHRAE---Great-Energy-Predictor-III-master\train_traget.csv')

In [909]:
gc.collect()

48